In [1]:
import env
import acquire
import split_scale
import pandas as pd
import numpy as np

In [2]:
df_iris = acquire.wrangle_iris()

   measurement_id  sepal_length  sepal_width  petal_length  petal_width  \
0               1           5.1          3.5           1.4          0.2   
1               2           4.9          3.0           1.4          0.2   
2               3           4.7          3.2           1.3          0.2   

   species_id  species_id species_name  
0           1           1       setosa  
1           1           1       setosa  
2           1           1       setosa  
(150, 8)
Index(['measurement_id', 'sepal_length', 'sepal_width', 'petal_length',
       'petal_width', 'species_id', 'species_id', 'species_name'],
      dtype='object')
measurement_id      int64
sepal_length      float64
sepal_width       float64
petal_length      float64
petal_width       float64
species_id          int64
species_id          int64
species_name       object
dtype: object
       measurement_id  sepal_length  sepal_width  petal_length  petal_width  \
count      150.000000    150.000000   150.000000    150.000000  

In [3]:
df_excel, df_excel_sample = acquire.excel_reader()

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents'], dtype='object')
customer_id     object
gender          object
partner         object
dependents      object
payment_type    object
churn           object
dtype: object
     is_senior_citizen  phone_service  internet_service  contract_type  \
min                0.0            0.0               0.0            0.0   
max                1.0            2.0               2.0            2.0   

     monthly_charges  total_charges  
min            18.25           18.8  
max           118.75         8684.8  


In [4]:
df_google, unique_categories = acquire.google_sheet()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int

yay


In [7]:
print('y')

y
